In [6]:
import requests
import pandas as pd
import geopandas as gpd
import geopandas as gpd
import pandas as pd
import requests
import geojson
from shapely.geometry import shape
import datetime
import numpy as np

In [7]:
%matplotlib inline
from matplotlib import pyplot as plt

In [8]:
def get_fish_popularity():  
    df_types=pd.read_csv(r'C:\Users\Intra24\Documents\hack_tourist\cdensity_types.csv') # only tourists
    df_types['date']=df_types['ts'].apply(lambda x: x.split(' ')[0])
    df_types['time']=df_types.ts.apply(lambda x: x.split(' ')[1])
    #df_types['median']=df_types.groupby('zid')['sum.2'].transform('median')
    fish=gpd.read_file(r"C:\Users\Intra24\Documents\hack_tourist\fishnet\fishnet.shp")
    fish_pop=fish.join(df_types.groupby('zid')['sum.2'].median(), how='left')
    fish_pop['sum.2'].fillna(0,inplace=True)
    fish_pop.rename(columns={'sum.2':'pop'}, inplace=True)
    fish_pop.to_csv('net_popular.csv')
    return fish_pop

In [9]:
fish=get_fish_popularity()

In [18]:
fish['pop_place']=fish['pop'].rank(method='first',ascending=False )

In [20]:
geo_attract=gpd.read_file('geo_attract.json')
attract_fish=gpd.sjoin(left_df=geo_attract, right_df=fish, op='intersects', how='inner')

C:\Users\Intra24\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [30]:
drop_capt=['Границы зон охраны объектов культурного наследия','Учреждения, осуществляющие управленческие функции','Объекты, вошедшие в программу дней исторического и культурного наследия',
       'Лауреаты конкурса Правительства Москвы «Московская реставрация»','Результаты контроля за состоянием благоустройства мест отдыха с купанием',
       'Объекты туристического показа и объекты гостиничного хозяйства города Москвы, приспособленные для лиц с ограниченными возможностями здоровья и людей пожилого возраста',
       'Туристский событийный календарь города Москвы на 2019 год','Границы территорий объектов культурного наследия',
       'Зимние активности парков, подведомственных Департаменту культуры города Москвы',
           'Места проведения фестиваля «Московское Лето. Цветочный Джем»',
       'Образовательные учреждения', 'Перечень видов животных, обитающих в Московском Зоопарке','Мероприятия, организованные при поддержке Департамента культуры города Москвы']

In [32]:
attract_fish=attract_fish[~attract_fish.Caption.isin(drop_capt)]

In [45]:
# меняет систему координат, чтобы перейти из градусов в метры и считает буфер радиусом 3км для каждого объекта
def calc_dist(df):
    df=df.to_crs(epsg='3857')
    df['radius']=df.geometry.buffer(3000)
    return df
attract_fish=calc_dist(attract_fish)

C:\Users\Intra24\Anaconda3\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


,properties,type,Caption,CategoryId,DepartmentId,dataset_id,global_id,Name,DisabilityFriendly,open_hour,...,3,4,5,6,geometry,index_right,Id,pop,pop_place,radius
8157,"{'DatasetId': 530, 'VersionNumber': 5, 'Releas...",Feature,Объекты культурного наследия,5,7,530,13780620.0,Станция Московского метрополитена Арбатско-Пок...,None,0,...,1,1,1,1,"POLYGON ((4202215.374 7516427.946, 4202267.444...",114437,114437,334.5,1.0,"POLYGON ((4199220.927 7516245.498, 4199217.460..."
29712,"{'DatasetId': 60622, 'VersionNumber': 1, 'Rele...",Feature,Залы спортивные,162,16,60622,904463258.0,Спортивный клуб «МАРК-АВРЕЛИЙ»,не приспособлен,8,...,1,1,1,1,POINT (4202150.402 7517076.484),114437,114437,334.5,1.0,"POLYGON ((4205150.402 7517076.484, 4205135.956..."
18926,"{'DatasetId': 1465, 'VersionNumber': 7, 'Relea...",Feature,Парковые территории,162,121,1465,4331764.0,Природно-исторический парк «Измайлово»,None,0,...,0,0,0,0,"POLYGON ((4212174.559 7520209.113, 4212216.178...",114437,114437,334.5,1.0,"POLYGON ((4198953.647 7514824.767, 4198951.126..."
19713,"{'DatasetId': 1465, 'VersionNumber': 7, 'Relea...",Feature,Парковые территории,162,121,1465,904087463.0,Сквер №2 напротив станции метро «Партизанская»,None,0,...,0,0,0,0,"POLYGON ((4202454.161 7516344.686, 4202449.902...",114437,114437,334.5,1.0,"POLYGON ((4199332.893 7516170.557, 4199319.599..."
29429,"{'DatasetId': 60622, 'VersionNumber': 1, 'Rele...",Feature,Залы спортивные,162,16,60622,904452101.0,Плоскостное спортивное сооружение «Туристско-г...,None,9,...,1,1,1,1,POINT (4202269.033 7516688.233),114437,114437,334.5,1.0,"POLYGON ((4205269.033 7516688.233, 4205254.587..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18919,"{'DatasetId': 1465, 'VersionNumber': 7, 'Relea...",Feature,Парковые территории,162,121,1465,4331744.0,Природно-исторический парк «Битцевский лес»,None,8,...,1,1,1,1,"MULTIPOLYGON (((4183389.693 7488112.789, 41841...",89108,89108,0.0,137501.0,"POLYGON ((4172793.051 7483624.089, 4172788.082..."
18921,"{'DatasetId': 1465, 'VersionNumber': 7, 'Relea...",Feature,Парковые территории,162,121,1465,4331751.0,Ландшафтный заказник «Тропарёвский»,None,8,...,1,1,1,1,"MULTIPOLYGON (((4169766.213 7492367.712, 41698...",95622,95622,0.0,140571.0,"POLYGON ((4166513.505 7492437.220, 4166520.737..."
18933,"{'DatasetId': 1465, 'VersionNumber': 7, 'Relea...",Feature,Парковые территории,162,121,1465,4331781.0,Природно-исторический парк «Москворецкий»,None,0,...,0,0,0,0,"POLYGON ((4162899.593 7524680.204, 4163150.819...",108641,108641,0.0,147085.0,"POLYGON ((4156955.236 7508928.879, 4156947.245..."
18933,"{'DatasetId': 1465, 'VersionNumber': 7, 'Relea...",Feature,Парковые территории,162,121,1465,4331781.0,Природно-исторический парк «Москворецкий»,None,0,...,0,0,0,0,"POLYGON ((4162899.593 7524680.204, 4163150.819...",108651,108651,0.0,147086.0,"POLYGON ((4156955.236 7508928.879, 4156947.245..."


In [46]:
attract_fish=attract_fish.sort_values('pop_place')

In [40]:
def in_radius(x, obj):
    try:
        return(x.within(obj))
    except:
        return False  
    #print(i,attract_fish_trans.at[i,"close_attractions"])

def get_iside_radius(obj):
    return(attract_fish[attract_fish.geometry.apply(lambda x:in_radius(x,obj))].index.tolist())

In [37]:
from tqdm import tqdm_notebook

In [64]:
## Find object within 3km radius for top-2000

In [47]:
dict_close=dict()
for ind in tqdm_notebook(attract_fish[:2000].index):
    l=get_iside_radius(attract_fish.loc[ind,'radius'])
    dict_close[ind]=l

TopologyException: side location conflict at 4193823.8531624423 7509852.4720845185
TopologyException: side location conflict at 4193823.8531624423 7509852.4720845185
TopologyException: side location conflict at 4193823.8531624423 7509852.4720845185
TopologyException: side location conflict at 4193823.8531624423 7509852.4720845185
TopologyException: side location conflict at 4193823.8531624423 7509852.4720845185
TopologyException: side location conflict at 4193823.8531624423 7509852.4720845185
TopologyException: side location conflict at 4193823.8531624423 7509852.4720845185
TopologyException: side location conflict at 4193823.8531624423 7509852.4720845185
TopologyException: side location conflict at 4193823.8531624423 7509852.4720845185
TopologyException: side location conflict at 4193823.8531624423 7509852.4720845185
TopologyException: side location conflict at 4193823.8531624423 7509852.4720845185
TopologyException: side location conflict at 4193823.8531624423 7509852.4720845185
Topo

TopologyException: side location conflict at 4193823.8531624423 7509852.4720845185
TopologyException: side location conflict at 4193823.8531624423 7509852.4720845185
TopologyException: side location conflict at 4193823.8531624423 7509852.4720845185
TopologyException: side location conflict at 4193823.8531624423 7509852.4720845185
TopologyException: side location conflict at 4193823.8531624423 7509852.4720845185
TopologyException: side location conflict at 4193823.8531624423 7509852.4720845185
TopologyException: side location conflict at 4193823.8531624423 7509852.4720845185
TopologyException: side location conflict at 4193823.8531624423 7509852.4720845185
TopologyException: side location conflict at 4193823.8531624423 7509852.4720845185
TopologyException: side location conflict at 4188861.5067722946 7515728.7358898381
TopologyException: side location conflict at 4188861.5067722946 7515728.7358898381
TopologyException: side location conflict at 4188861.5067722946 7515728.7358898381
Topo

In [49]:
import json

In [144]:
with open('close_attract.json', 'w') as f:
    json.dump(dict_close, f)

In [59]:
attract_fish.drop(['properties','type'], axis=1, inplace=True)

In [63]:
attract_fish[attract_fish.columns[:-1]].reset_index().to_file('attract_fish1.csv', driver='CSV')

In [69]:
attract_fish.Caption.unique()

array(['Объекты культурного наследия', 'Залы спортивные',
       'Парковые территории',
       'Аттракционы в торговых центрах и комплексах', 'Библиотеки',
       'Спортивные площадки', 'Места для досуга и отдыха с детьми',
       'Музеи', 'Археологические находки на реконструируемых улицах',
       'Интерактивная карта учреждений культуры города Москвы',
       'Мемориальные доски города Москвы',
       'Перечень пешеходных туристических маршрутов Москвы', 'Театры',
       'Концертные организации', 'Аттракционы в скверах и парках',
       'Перечень автобусных туристических маршрутов Москвы',
       'Бассейны плавательные крытые',
       'Перечень смотровых площадок, оборудованных стационарными биноклями',
       'Площадки пейнтбольные', 'Кинотеатры', 'Дома культуры и клубы',
       'Залы тренажерные', 'Тренажерные городки (воркауты)',
       'Поля футбольные', 'Открытые ледовые катки',
       'Лыжные трассы города Москвы',
       'Перечень скульптур под открытым небом, расположенных н

In [71]:
df_cat=pd.read_csv(open(r'C:\Users\Intra24\Documents\hack_tourist\сategories_done1.csv','r'))

In [86]:
df_cat=df_cat.rename(columns={'Column1':'Caption','Season (winter, autumn, spring, summer, all)':'season', 'Morning, afternoon, evening':'day_part','Free or paid':'is_paid','Time, h':'hours','Inside/ outside':'where'})#.drop(['Column2'], axis=1)

In [164]:
df_attract=attract_fish.reset_index()[['global_id','Caption','Name','Id','pop','pop_place','geometry','index']].merge(df_cat, on='Caption')

In [140]:
df_attract['Category_2'].unique()

array(['Архитектура', 'Досуг', 'Библиотеки', 'Фитнес', 'Музеи', 'Театры',
       'Концерты', 'Аттракционы', 'Водный спорт', 'Командные игры',
       'Кино', 'Культура', 'Зимний спорт', 'Авто', 'Выставки',
       'Образование', 'Другое', 'Стрельбище'], dtype=object)

In [138]:
df_attract.rename(columns={'Id':'fish_id'}, inplace=True)
#df_attract.to_file('fin_attract.csv', driver='CSV')

In [92]:
df_izi=pd.read_csv(open(r'C:\Users\Intra24\Documents\hack_tourist\mus_izi.csv','r'))

In [99]:
df_izi.rename(columns={'Ссылка на аудиогид':'link_izi'}, inplace=True)

In [101]:
df_izi_full=df_izi.merge(df_attract[['Name','index']], on='Name')[['index','Name','link_izi']]

In [102]:
df_izi_full.to_csv('df_izi_full.csv')

In [147]:
df_attract[['global_id','hours']].to_csv('glob_id_hour.csv')

In [165]:
df_un_attract=df_attract.groupby(['Name','Category','Category_2','global_id','index'])['pop_place'].median().reset_index()

In [174]:
df_un_attract=df_un_attract[df_un_attract['Name']!='Дворовая территория']
df_un_attract=df_un_attract[df_un_attract['Name']!='Спортивный комплекс образовательного учреждения']
df_un_attract.to_csv('list_attract_pop.csv')

In [159]:
close_glid=dict(zip(attract_fish.loc[list(dict_close.keys())]['global_id'].values.tolist(), list(dict_close.values())))

In [166]:
close_glid[13780620.0]

[8157,
 29712,
 19713,
 29429,
 652,
 19713,
 2580,
 19524,
 2796,
 1705,
 7286,
 18602,
 18939,
 4402,
 4805,
 29963,
 17276,
 15643,
 28355,
 13996,
 15787,
 19647,
 18939,
 15738,
 7286,
 17334,
 16122,
 30021,
 25059,
 28502,
 25449,
 14819,
 16100,
 14018,
 18579,
 17110,
 15275,
 15589,
 13227,
 16152,
 2796,
 2580,
 7286,
 18939,
 4805,
 28520,
 17907,
 13264,
 348,
 17335,
 20133,
 25544,
 17510,
 15709,
 15314,
 14105,
 13396,
 29446,
 30287,
 30290,
 29442,
 13402,
 13341,
 18436,
 18435,
 18566,
 18434,
 20568,
 18864,
 20505,
 23548,
 14071,
 18328,
 8619,
 8620,
 14007,
 8650,
 29431,
 17949,
 29432,
 13425,
 13424,
 13423,
 18331,
 24638,
 23547,
 1299,
 29910,
 29909,
 29822,
 29985,
 20424,
 17283,
 489,
 15317,
 14204,
 17645,
 15368,
 13992,
 25888,
 25922,
 17497,
 17886,
 482,
 25684,
 25535,
 28704,
 15653,
 24498,
 20921,
 3254,
 3158,
 19879,
 18607,
 20627,
 20509,
 20155,
 18437,
 20720,
 16201,
 18112,
 18439,
 18939,
 20779,
 652,
 1371,
 1708,
 1823,
 26099,

In [162]:
with open('close_attract_glid.json', 'w') as f:
    json.dump(close_glid, f)

In [126]:
df_un_attract[df_un_attract.groupby(['Category_2'])['pop_place'].rank()<3].reset_index()

,level_0,Name,Category,Category_2,index,pop_place
0,3000,"Carousel 6,70",Парки,Аттракционы,295,31.0
1,3006,Carousel DOUBLE-DECK - PARK MODEL,Парки,Аттракционы,280,9.0
2,3113,Автодром «Картинг-клуб»,Спорт,Авто,18602,71.0
3,3205,Ансамбль духовной музыки «Благовест»,Культура,Концерты,601,12.0
4,3661,Выставочный зал «Солянка ВПА»,Культура,Выставки,588,156.0
5,3681,Галерея А3,Культура,Выставки,599,92.0
6,4100,Государственное автономное учреждение культуры...,С детьми,Образование,16202,562.0
7,4231,Дворовая территория,Спорт,Другое,16374,263.0
8,4249,Дворовая территория,Спорт,Командные игры,13044,21.0
9,4451,Дворовая территория,Спорт,Фитнес,13855,8.0


In [182]:
list_obj=attract_fish.iloc[:4]['global_id'].values.tolist()

In [223]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from pyroutelib3 import Router

def build_route(lat1, lon1, lat2, lon2, type_move='foot'):
    router = Router('foot') # car, cycle, foot, horse, tram, train

    start = router.findNode(lat1, lon1) # Find start and end nodes
    end = router.findNode(lat2, lon2)
    status, route = router.doRoute(start, end) # Find the route - a list of OSM nodes
    print(status)
    if status == 'success':
        route = list(map(router.nodeLatLon, route)) # Get actual route coordinates
    return  route

def calculate_distance_meters(route):
    total = 0.0
    for idx in range(1, len(route)):
        total += router.distance(route[idx - 1], route[idx])
    return total*1000

In [187]:
l=len(list_obj)
m_dist=np.zeros((l,l))

In [199]:
geo_attract.geometry.centroid.x

0        37.820872
1        37.178656
2        37.696268
3        37.733018
4        37.595020
           ...    
30981    37.364117
30982    37.374580
30983    37.541060
30984    37.499551
30985    37.492514
Length: 30986, dtype: float64

In [220]:
geo_attract.loc[geo_attract['global_id']==obj1,'geometry'].centroid.x.values[0]

37.74938665005468

In [244]:
geo_attract['type'].values

array(['Feature', 'Feature', 'Feature', ..., 'Feature', 'Feature',
       'Feature'], dtype=object)

In [242]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.reverse("55.7473, 37.6051")
print(location.address)

Музей изобразительных искусств им. Пушкина, 12 с1, улица Волхонка, 31, Старое Ваганьково, Москва, Центральный административный округ, Москва, Центральный федеральный округ, 119019, Россия


In [250]:
attract_fish['geometry'].centroid.distance(attract_fish.loc[geo_attract['global_id']==obj1,'geometry'].centroid).isnull().mean()

0.9999604946075139

In [251]:
attract_fish['geometry'].centroid.distance()

8157     POINT (4202242.500 7516411.644)
29712    POINT (4202150.402 7517076.484)
18926    POINT (4206768.976 7514438.231)
19713    POINT (4202404.991 7516423.712)
29429    POINT (4202269.033 7516688.233)
                      ...               
18919    POINT (4181136.626 7481532.792)
18921    POINT (4169925.795 7489239.355)
18933    POINT (4166146.287 7513140.763)
18933    POINT (4166146.287 7513140.763)
18933    POINT (4166146.287 7513140.763)
Length: 25313, dtype: geometry

In [261]:
list_obj=attract_fish.loc[dict_close[8157][0:3]]['global_id'].values.tolist()

In [252]:
attract_fish

,Caption,CategoryId,DepartmentId,dataset_id,global_id,Name,DisabilityFriendly,open_hour,close_hour,0,...,3,4,5,6,geometry,index_right,Id,pop,pop_place,radius
8157,Объекты культурного наследия,5,7,530,13780620.0,Станция Московского метрополитена Арбатско-Пок...,None,0,24,1,...,1,1,1,1,"POLYGON ((4202215.374 7516427.946, 4202267.444...",114437,114437,334.5,1.0,"POLYGON ((4199220.927 7516245.498, 4199217.460..."
29712,Залы спортивные,162,16,60622,904463258.0,Спортивный клуб «МАРК-АВРЕЛИЙ»,не приспособлен,8,20,1,...,1,1,1,1,POINT (4202150.402 7517076.484),114437,114437,334.5,1.0,"POLYGON ((4205150.402 7517076.484, 4205135.956..."
18926,Парковые территории,162,121,1465,4331764.0,Природно-исторический парк «Измайлово»,None,0,23,0,...,0,0,0,0,"POLYGON ((4212174.559 7520209.113, 4212216.178...",114437,114437,334.5,1.0,"POLYGON ((4198953.647 7514824.767, 4198951.126..."
19713,Парковые территории,162,121,1465,904087463.0,Сквер №2 напротив станции метро «Партизанская»,None,0,23,0,...,0,0,0,0,"POLYGON ((4202454.161 7516344.686, 4202449.902...",114437,114437,334.5,1.0,"POLYGON ((4199332.893 7516170.557, 4199319.599..."
29429,Залы спортивные,162,16,60622,904452101.0,Плоскостное спортивное сооружение «Туристско-г...,None,9,18,1,...,1,1,1,1,POINT (4202269.033 7516688.233),114437,114437,334.5,1.0,"POLYGON ((4205269.033 7516688.233, 4205254.587..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18919,Парковые территории,162,121,1465,4331744.0,Природно-исторический парк «Битцевский лес»,None,8,22,1,...,1,1,1,1,"MULTIPOLYGON (((4183389.693 7488112.789, 41841...",89108,89108,0.0,137501.0,"POLYGON ((4172793.051 7483624.089, 4172788.082..."
18921,Парковые территории,162,121,1465,4331751.0,Ландшафтный заказник «Тропарёвский»,None,8,22,1,...,1,1,1,1,"MULTIPOLYGON (((4169766.213 7492367.712, 41698...",95622,95622,0.0,140571.0,"POLYGON ((4166513.505 7492437.220, 4166520.737..."
18933,Парковые территории,162,121,1465,4331781.0,Природно-исторический парк «Москворецкий»,None,0,23,0,...,0,0,0,0,"POLYGON ((4162899.593 7524680.204, 4163150.819...",108641,108641,0.0,147085.0,"POLYGON ((4156955.236 7508928.879, 4156947.245..."
18933,Парковые территории,162,121,1465,4331781.0,Природно-исторический парк «Москворецкий»,None,0,23,0,...,0,0,0,0,"POLYGON ((4162899.593 7524680.204, 4163150.819...",108651,108651,0.0,147086.0,"POLYGON ((4156955.236 7508928.879, 4156947.245..."


In [262]:
list_obj

[13780620.0, 904463258.0, 904087463.0, 904087463.0]

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [266]:
for i,obj1 in enumerate(list_obj):
    for j,obj2 in enumerate(list_obj):
        if i!=j:
#             m_dist[i,j]= geo_attract.loc[geo_attract['global_id']==obj1,'geometry'].distance(geo_attract.loc[geo_attract['global_id']==obj2,'geometry'])
            lon1=geo_attract.loc[geo_attract['global_id']==obj1,'geometry'].centroid.x.values[0]
            lat1=geo_attract.loc[geo_attract['global_id']==obj1,'geometry'].centroid.y.values[0]
            lon2=geo_attract.loc[geo_attract['global_id']==obj2,'geometry'].centroid.x.values[0]
            lat2=geo_attract.loc[geo_attract['global_id']==obj2,'geometry'].centroid.y.values[0]
            route=build_route(lat1, lon1, lat2, lon2, type_move='foot')
            m_dist[i,j]=calculate_distance_meters(route)

no_route
no_route
no_route


URLError: <urlopen error [WinError 10065] A socket operation was attempted to an unreachable host>

In [245]:
day=[9,10,11,12,13,14,15,16,17,18,19]

In [246]:
obj=[12,13]

In [249]:
day=[h for h in day if (h<=obj[0]-2)or (h>=obj[1]+1)]

[9, 10, 14, 15, 16, 17, 18, 19]

In [ ]:
obj1=[day[0],day[1]]
obj2=[day[2],day[3]]